In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import numpy as np
import pandas as pd

import pickle
import jieba
import h5py

SOS_token = 0
EOS_token = 1
__PADDING__ = 2
lang1 = 'en'
lang2 = 'zh'

In [ ]:
class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS",2:"__PADDING__"}
        self.n_words = 3  # Count SOS and EOS and __PADDING__
        
    def countWordFreq(self, word):
        if word not in self.word2count:
            self.word2count[word] = 1
            self.n_words += 1
        else:
            self.word2count[word] +=1

    #通过不断输入sentence（字符串的格式），构建词与下标的对应（词典），方便制作one-hot。
    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)
            
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1
            
    def save(self, path):
        with open(path,'wb') as f:
            pickle.dump([self.name,self.word2index, self.word2count, self.index2word, self.n_words],f)
    
    def load(self,path):
        with open(path,'rb') as f:
            name, self.word2index, self.word2count, self.index2word, self.n_words = pickle.load(f)
        if self.name != name:
            print('error: Name error------------------------------!')
            
    def indexesFromSentence(self, sentence):
        indexes = []
        all_lang_keys = self.word2index.keys()
        for word in sentence.split(' '):
            if word in all_lang_keys:
                indexes.append(self.word2index[word])
        indexes.append(EOS_token)
        return indexes
            
##################################################################


In [ ]:
##open data file 
en_lines = open('../data/train.%s'% lang1).read().strip().split('\n')
zh_lines = open('../data/train.%s'% lang2).read().strip().split('\n')
data_length = len(zh_lines)
random_sort_sequence = random.sample(range(data_length),data_length)

random_sort_sequence = random_sort_sequence[0:100]##take 100 sentences for test .
    

In [ ]:
inputlang = Lang('en')
outputlang = Lang('zh')

for index in random_sort_sequence:
    tmp = [normalizeString(s) for s in en_lines[index].split('\t')]
    
    

In [3]:
inputlang = Lang('en')
outputlang = Lang('zh')
inputlang.load('../data/inputlang.pkl')
outputlang.load('../data/outputlang.pkl')


In [4]:
h5py_file = h5py.File('../data/train_pairs.h5py','r')
pairs = h5py_file['pairs']

In [5]:
print(pairs[0][0],pairs[0][1].decode('gb2312'))

b'not having made me the first man '  但 他 并 没 有 让 我 成 为 你 第 一 个 男 人 ， 


In [19]:
en_MAX_LENGTH = 50
zh_MAX_LENGTH = 80
en_lengths = []
zh_lengths = []
for pair in pairs:
    try:
        input_sent = pair[0].decode('utf-8')
        output_sent = pair[1].decode('gb2312')
        en_index = inputlang.indexesFromSentence(input_sent)
        zh_index = outputlang.indexesFromSentence(output_sent)
        zh_index.insert(0,2)#decoder 的首位置添加识别符号
        if len(en_index) > en_MAX_LENGTH or len(zh_index) > zh_MAX_LENGTH:
            continue
        en_lengths.append(len(en_index))
        zh_lengths.append(len(zh_index))
        
    except UnicodeDecodeError:
        print('Unicode decode error and the number is: ')


[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 1]

In [20]:
print(len(zh_index))

17
